## Setup
Universe: Nifty FnO  
Positional:   
- Entry Time - 09:20 Friday Close  
- Exit Time - Expiry day 15:15

## Rules

Short ITM - 1 -> Call option  
Short OTM - 1 -> Put Option  
SL -> 25% of Entry Price

----

### Importing Libraries

In [9]:
import pandas as pd
import numpy as np
import openpyxl
import pandas_ta as ta
import sys
sys.path.append (r"C:\TheG\codes\codes")
import self_code as sc
import Utils as ut
import glob
import warnings
warnings.filterwarnings("ignore")

### Importing Main Dataset

In [10]:
# importing the dataset of nifty50 index

nifty = pd.read_csv(r"C:\TheG\update\NIFTY.csv")
nifty.head(3)

,datetime,open,high,low,close,symbol,volume
0,03-01-2019 09:15,10796.80,10799.5,10770.00,10770.0,NIFTY,0
1,03-01-2019 09:16,10771.00,10771.0,10755.25,10759.4,NIFTY,0
2,03-01-2019 09:17,10760.45,10773.1,10759.60,10773.1,NIFTY,0


### Data Processing

In [11]:
# resampling the dataframes to 1 minute and having only datapoint of 09:20
nifty = sc.resample(nifty, '1min', t1 = '09:20:00', t2 = '09:20:00')

# getting day of the week
nifty.reset_index(inplace = True)
nifty['day'] = nifty['datetime'].dt.weekday
nifty = nifty.set_index('datetime')

# removing every day except friday
nifty = nifty[nifty['day'] == 4]
nifty.head(3)

,open,high,low,close,volume,day
datetime,,,,,,
2019-01-04 09:20:00,10721.50,10724.70,10720.3,10723.15,0,4
2019-01-11 09:20:00,10841.20,10850.15,10841.2,10848.40,0,4
2019-01-18 09:20:00,10915.35,10915.70,10900.9,10902.40,0,4


### Understanding Heatmaps for Optimisation

In [12]:
# pnl is the tradelog dataframe where we'll store tradelog

pnl = pd.DataFrame(columns = ['Type', 'Entry Price', 'Entry Time', 'Exit Price', 'Exit Time', 'Points', 'Pnl'])

# starting the loop to iterate one by one over the nifty dataset
for row, value in nifty.iterrows():
    
    # getting the ticker value of 09:20 open at friday to get ATM
    symbol_ce = ut.prepareWeeklyOptionsSymbol('NIFTY', ut.priceToATM(value['open'], 'NIFTY'), 'CE', row) # this funcation gives ticker name.. Example:- 'NIFTY17JUL9900CE'
    symbol_pe = ut.prepareWeeklyOptionsSymbol('NIFTY', ut.priceToATM(value['open'], 'NIFTY'), 'PE', row)
    
    try:
        # getting the dataset of this particular symbol
        ce = pd.read_csv(r"C:\TheG\update\nifty options\{}.csv".format(symbol_ce))
        pe = pd.read_csv(r"C:\TheG\update\nifty options\{}.csv".format(symbol_pe))
    except Exception as e:
        # if we dont find this symbol, continue
        continue
    
    # variable declaration and reset
    status = 0
    entry_price = 0
    entry_time = 0
    exit_price = 0
    exit_time = 0
    # iterating over the call option fro backtesting
    for idx, val in ce.iterrows():
        
        # we'll enter when ce row data datetime is as same as current datetime     - Entry Condition for call option
        if status == 0 and val['Date'] == str(row.date()) and val['Time'] == '09:20:59':
            status = 1
            entry_price = val['Open']
            entry_time = val['Date'] + ' ' + val['Time']
            continue
        # condition for sl hit
        elif status == 1 and val['High'] >= 1.25 * entry_price >= val['Low']:
            exit_price = 1.25 * entry_price
            exit_time = val['Date'] + ' ' + val['Time']
            pnl.loc[len(pnl)] = [symbol_ce, entry_price, entry_time, exit_price, exit_time, entry_price - exit_price, (entry_price - exit_price) * 50]
            break
        # condition for gap up of option price and sl slips 
        elif status == 1 and val['Open'] >= 1.25 * entry_price:
            exit_price = val['Open']
            exit_time = val['Date'] + ' ' + val['Time']
            pnl.loc[len(pnl)] = [symbol_ce, entry_price, entry_time, exit_price, exit_time, entry_price - exit_price, (entry_price - exit_price) * 50]
            break
        # final exit at expiry day at 15:25 open
        elif status == 1 and ce.iloc[-5]['Date'] == val['Date'] and ce.iloc[-5]['Time'] == val['Time']:
            exit_price = val['Open']
            exit_time = val['Date'] + ' ' + val['Time']
            pnl.loc[len(pnl)] = [symbol_ce, entry_price, entry_time, exit_price, exit_time, entry_price - exit_price, (entry_price - exit_price) * 50]
            break
    
    ############################# SIMILAR CODE FOR PUT SIDE ####################################
    # variable declaration and reset
    status = 0
    entry_price = 0
    entry_time = 0
    exit_price = 0
    exit_time = 0
    # iterating over the call option fro backtesting
    for idx, val in pe.iterrows():
        
        # we'll enter when pe row data datetime is as same as current datetime     - Entry Condition for call option
        if status == 0 and val['Date'] == str(row.date()) and val['Time'] == '09:20:59':
            status = 1
            entry_price = val['Open']
            entry_time = val['Date'] + ' ' + val['Time']
            continue
        # condition for sl hit
        elif status == 1 and val['High'] >= 1.25 * entry_price >= val['Low']:
            exit_price = 1.25 * entry_price
            exit_time = val['Date'] + ' ' + val['Time']
            pnl.loc[len(pnl)] = [symbol_pe, entry_price, entry_time, exit_price, exit_time, entry_price - exit_price, (entry_price - exit_price) * 50]
            break
        # condition for gap up of option price and sl slips 
        elif status == 1 and val['Open'] >= 1.25 * entry_price:
            exit_price = val['Open']
            exit_time = val['Date'] + ' ' + val['Time']
            pnl.loc[len(pnl)] = [symbol_pe, entry_price, entry_time, exit_price, exit_time, entry_price - exit_price, (entry_price - exit_price) * 50]
            break
        # final exit at expiry day at 15:25 open
        elif status == 1 and pe.iloc[-5]['Date'] == val['Date'] and pe.iloc[-5]['Time'] == val['Time']:
            exit_price = val['Open']
            exit_time = val['Date'] + ' ' + val['Time']
            pnl.loc[len(pnl)] = [symbol_pe, entry_price, entry_time, exit_price, exit_time, entry_price - exit_price, (entry_price - exit_price) * 50]
            break

In [7]:
pnl.head(4)

,Type,Entry Price,Entry Time,Exit Price,Exit Time,Points,Pnl
0,NIFTY18AUG11550CE,79.30,2018-08-24 09:20:59,99.1250,2018-08-27 09:15:59,-19.8250,-991.250
1,NIFTY18AUG11550PE,48.25,2018-08-24 09:20:59,60.3125,2018-08-24 12:52:59,-12.0625,-603.125
2,NIFTY18SEP11300CE,101.20,2018-09-21 09:20:59,0.0500,2018-09-27 15:25:59,101.1500,5057.500
3,NIFTY18SEP11300PE,48.75,2018-09-21 09:20:59,60.9375,2018-09-21 11:53:59,-12.1875,-609.375


### Metric Generation

In [13]:
# converting series to datetime like values for metric generation

try:
    pnl['Entry Time']= pd.to_datetime(pnl['Entry Time'] ,format='%d-%m-%Y %H:%M:%S')
    pnl['Exit Time']= pd.to_datetime(pnl['Exit Time'] ,format='%d-%m-%Y %H:%M:%S')
except:
    pnl['Entry Time']= pd.to_datetime(pnl['Entry Time'] ,format='%Y-%m-%d %H:%M:%S')
    pnl['Exit Time']= pd.to_datetime(pnl['Exit Time'] ,format='%Y-%m-%d %H:%M:%S')

In [14]:
sc.final(pnl, r'C:\work\balysis\Youtube\Trading Strategies\The 09-20 Positional\result.xlsx', capital = 150000)